<a href="https://colab.research.google.com/github/Rupam-01/Finetuning_using_Distilbert/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
df = messages = pd.read_csv('training_data_extended.tsv',sep='\t',names=["label","text"])

In [67]:
df.head()

,label,text
0,valid,Raising funds to purchase school uniforms and ...
1,valid,Need funds to build fencing for a wildlife reh...
2,valid,Raising funds to purchase school uniforms and ...
3,invalid,Need funds to buy a sports car.
4,invalid,Want to impress my friends with a yacht trip.


In [68]:
df.shape

(3199, 2)

In [69]:
X = list(df['text'])

In [74]:
y=list(df['label'])

In [75]:
y = list(pd.get_dummies(y, drop_first=True)['valid'])

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size=0.20,random_state =0)

In [112]:
!pip install transformers


In [113]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset


In [114]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [118]:
import numpy as np
y_train = np.array(y_train).astype(int)
y_test = np.array(y_test).astype(int)


In [119]:
# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = CustomDataset(train_encodings, y_train)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    #evaluation_strategy="epoch",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train
trainer.train()

# Evaluate
eval_results = trainer.evaluate()
print("Evaluation:", eval_results)

# Predict
predictions = trainer.predict(test_dataset)
print("Predicted logits shape:", predictions.predictions.shape)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.729100
20,0.698700
30,0.665000
40,0.584000
50,0.541900
60,0.440800
70,0.350500
80,0.230800
90,0.225200
100,0.162900
